In [ ]:
#!/usr/bin/env python3
"""
predict.py (robust final)

- Uses 5m bars (60d) by default for long history.
- Defensive feature-building and multi-step return targets.
- Trains MultiOutputRegressor on returns and enters realtime loop.
- Robust access to last_price using .iloc and try/except so the loop won't crash.
"""

import os
import time
from datetime import datetime, timezone
import traceback

import joblib
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

# --------------- CONFIG ----------------
TICKER = "ASHOKLEY.BO"
INTERVAL = "5m"
PERIOD = "60d"
WINDOW = 30
K = 12
REFRESH_SECONDS = 60
MODEL_FILE = "model_returns_k.joblib"
SCALER_FILE = "scaler_returns_k.joblib"
LOG_FILE = "logs/predictions_returns.csv"
DEBUG_BAD = "debug_bad_rows.npy"
os.makedirs("logs", exist_ok=True)
# ---------------------------------------

def fetch_history(ticker: str, interval: str, period: str) -> pd.DataFrame:
    def process_df(df):
        if df is None or df.empty:
            return pd.DataFrame()
        if not isinstance(df.index, pd.DatetimeIndex):
            df.index = pd.to_datetime(df.index)
        cols = [c for c in ["Open","High","Low","Close","Volume"] if c in df.columns]
        df = df.loc[:, cols]
        for c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
        df.dropna(inplace=True)
        return df

    try:
        df = yf.download(tickers=ticker, interval=interval, period=period,
                         progress=False, threads=False, auto_adjust=False)
        return process_df(df)
    except Exception as e:
        print("Fetch error:", e)
        # fallback minimal handling for 1m
        if interval == "1m":
            try:
                df = yf.download(tickers=ticker, interval="1m", period="7d",
                                 progress=False, threads=False, auto_adjust=False)
                return process_df(df)
            except Exception:
                try:
                    df = yf.download(tickers=ticker, interval="5m", period=period,
                                     progress=False, threads=False, auto_adjust=False)
                    return process_df(df)
                except Exception as e3:
                    print("All fetch fallbacks failed:", e3)
                    return pd.DataFrame()
        return pd.DataFrame()

def _is_sequence_like(x):
    if isinstance(x, (list, tuple, np.ndarray, pd.Series)):
        return True
    try:
        import collections.abc
        if isinstance(x, collections.abc.Sequence) and not isinstance(x, (str, bytes)):
            return True
    except Exception:
        pass
    return False

def _maybe_extract_scalar(x):
    try:
        if isinstance(x, pd.Series):
            a = x.values
        else:
            a = np.asarray(x)
        if a.ndim == 0:
            return a.item()
        if a.size == 1:
            return a.flatten()[0]
    except Exception:
        pass
    return x

def build_X_y_returns_safe(df: pd.DataFrame, window: int, k: int):
    closes = df["Close"].values
    volumes = df["Volume"].values
    idx = df.index
    n = len(df)
    expected_len = 2 * window + 4

    X_rows = []
    y_rows = []
    bad_rows = []

    for i in range(window, n - k):
        try:
            past_closes = closes[i - window:i]
            if len(past_closes) != window:
                bad_rows.append((i, "past_closes_len", len(past_closes)))
                continue
            past_returns = (past_closes[1:] - past_closes[:-1]) / (past_closes[:-1] + 1e-9)
            if len(past_returns) != (window - 1):
                bad_rows.append((i, "past_returns_len", len(past_returns)))
                continue

            feats = []
            feats.extend(list(past_closes))
            feats.extend(list(past_returns))
            feats.append(np.mean(past_closes))
            feats.append(np.std(past_closes))
            feats.append(volumes[i - 1])
            dt = idx[i]
            minute_of_day = dt.hour * 60 + dt.minute
            feats.append(np.sin(2 * np.pi * minute_of_day / 1440))
            feats.append(np.cos(2 * np.pi * minute_of_day / 1440))

            # ensure scalar elements
            safe_feats = []
            malformed = False
            for j, el in enumerate(feats):
                if _is_sequence_like(el):
                    el2 = _maybe_extract_scalar(el)
                    if _is_sequence_like(el2):
                        bad_rows.append((i, f"elem_multi_at_{j}", type(el2).__name__))
                        malformed = True
                        break
                    else:
                        safe_feats.append(el2)
                else:
                    safe_feats.append(el)

            if malformed:
                continue

            # cast to float array
            try:
                feat_arr = np.asarray(safe_feats, dtype=float)
            except Exception as e:
                bad_rows.append((i, f"cast_error:{e}", str(safe_feats[:8])))
                continue

            if feat_arr.ndim != 1 or feat_arr.shape[0] != expected_len:
                bad_rows.append((i, f"bad_shape:{feat_arr.shape}", feat_arr.shape))
                continue

            future = closes[i:i+k]
            if len(future) != k:
                bad_rows.append((i, "future_len", len(future)))
                continue

            returns_k = (future - closes[i]) / (closes[i] + 1e-9)
            if np.any(np.isnan(returns_k)):
                bad_rows.append((i, "target_nan", returns_k))
                continue

            # append as 2D rows to stack safely later
            X_rows.append(feat_arr.reshape(1, -1))
            y_rows.append(np.asarray(returns_k, dtype=float).reshape(1, k))

        except Exception as ex:
            bad_rows.append((i, f"exception:{ex}", traceback.format_exc()))
            continue

    # reporting
    print(f"[build] total rows={n}, valid={len(X_rows)}, bad={len(bad_rows)}")
    if bad_rows:
        print("Examples of bad rows (up to 8):")
        for b in bad_rows[:8]:
            print(" ", b[0], "-", b[1], "-", str(b[2])[:160])
        try:
            np.save(DEBUG_BAD, bad_rows, allow_pickle=True)
            print(f"Wrote bad rows to {DEBUG_BAD}")
        except Exception as e:
            print("Could not save debug bad rows:", e)

    if len(X_rows) == 0:
        return np.empty((0, expected_len)), np.empty((0, k))

    X = np.vstack(X_rows)
    y = np.vstack(y_rows)
    return X, y

def train_and_save(X, y):
    print("Training model: shapes X,y =", X.shape, y.shape)
    if X.shape[0] != y.shape[0]:
        raise RuntimeError(f"Sanity check failed: X samples={X.shape[0]} y samples={y.shape[0]}")
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, shuffle=False)
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_val_s = scaler.transform(X_val)
    base = GradientBoostingRegressor(n_estimators=200, learning_rate=0.1, max_depth=3)
    model = MultiOutputRegressor(base, n_jobs=-1)
    model.fit(X_train_s, y_train)
    y_pred = model.predict(X_val_s)
    mae = mean_absolute_error(y_val, y_pred)
    print(f"Validation MAE (returns avg): {mae:.6f}")
    joblib.dump(model, MODEL_FILE)
    joblib.dump(scaler, SCALER_FILE)
    print("Saved model & scaler.")
    return model, scaler

def load_if_present():
    if os.path.exists(MODEL_FILE) and os.path.exists(SCALER_FILE):
        model = joblib.load(MODEL_FILE)
        scaler = joblib.load(SCALER_FILE)
        print("Loaded model and scaler from disk.")
        return model, scaler
    return None, None

def make_feature_vector_from_recent(df_recent, window):
    if len(df_recent) < window:
        return None
    closes = df_recent["Close"].values
    volumes = df_recent["Volume"].values
    past_closes = closes[-window:]
    past_returns = (past_closes[1:] - past_closes[:-1]) / (past_closes[:-1] + 1e-9)
    feats = []
    feats.extend(list(past_closes))
    feats.extend(list(past_returns))
    feats.append(np.mean(past_closes))
    feats.append(np.std(past_closes))
    feats.append(volumes[-1])
    dt = df_recent.index[-1]
    minute_of_day = dt.hour * 60 + dt.minute
    feats.append(np.sin(2 * np.pi * minute_of_day / 1440))
    feats.append(np.cos(2 * np.pi * minute_of_day / 1440))
    # sanitize elements
    safe_feats = []
    for el in feats:
        if _is_sequence_like(el):
            el2 = _maybe_extract_scalar(el)
            if _is_sequence_like(el2):
                return None
            safe_feats.append(el2)
        else:
            safe_feats.append(el)
    try:
        return np.asarray(safe_feats, dtype=float).reshape(1, -1)
    except Exception:
        return None

def log_preds(ts, ticker, preds_returns, last_price):
    row = {"timestamp": ts, "ticker": ticker, "last_price": float(last_price)}
    for i, r in enumerate(preds_returns, start=1):
        row[f"pred_ret_t+{i}"] = float(r)
        row[f"pred_price_t+{i}"] = float(last_price * (1 + r))
    df = pd.DataFrame([row])
    header = not os.path.exists(LOG_FILE)
    df.to_csv(LOG_FILE, mode="a", header=header, index=False)

def main():
    print("Downloading history for training...")
    df_hist = fetch_history(TICKER, INTERVAL, PERIOD)
    if df_hist.empty:
        print("No history returned. Exiting.")
        return
    print("History rows:", len(df_hist))

    X, y = build_X_y_returns_safe(df_hist, WINDOW, K)
    print("Built X,y shapes:", X.shape, y.shape)
    if X.size == 0:
        print("No valid samples after cleaning. Exiting.")
        return

    model, scaler = load_if_present()
    if model is None or scaler is None:
        model, scaler = train_and_save(X, y)

    print("Realtime loop (Ctrl+C to stop).")
    try:
        while True:
            df_now = fetch_history(TICKER, INTERVAL, "2d")
            if df_now.empty or len(df_now) < WINDOW:
                print("Not enough recent data; sleeping...")
                time.sleep(REFRESH_SECONDS)
                continue

            # Build feature vector safely
            fv = make_feature_vector_from_recent(df_now, WINDOW)
            if fv is None:
                print("Could not build safe feature vector; sleeping...")
                time.sleep(REFRESH_SECONDS)
                continue

            # Safely get last price using iloc and try/except to avoid rare pandas oddities
            try:
                last_price = float(df_now.iloc[-1]["Close"])
            except Exception as e:
                print("Warning: failed to read last_price (skipping this iteration):", e)
                time.sleep(REFRESH_SECONDS)
                continue

            # Predict
            try:
                fv_s = scaler.transform(fv)
                preds = model.predict(fv_s)[0]  # length K
            except Exception as e:
                print("Warning: model prediction failed this cycle:", e)
                time.sleep(REFRESH_SECONDS)
                continue

            now = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S %Z")
            print("="*60)
            print(f"[{now}] Last close = {last_price:.4f}")
            for i, r in enumerate(preds, start=1):
                p_price = last_price * (1 + r)
                print(f" t+{i:02d}: ret={r*100:+.3f}% -> price ≈ {p_price:.4f}")
            log_preds(now, TICKER, preds, last_price)
            time.sleep(REFRESH_SECONDS)
    except KeyboardInterrupt:
        print("\nStopped by user. Exiting.")

if __name__ == "__main__":
    main()


History rows: 4377
[build] total rows=4377, valid=4335, bad=0
Built X,y shapes: (4335, 64) (4335, 12)
Loaded model and scaler from disk.
Realtime loop (Ctrl+C to stop).


C:\Users\chara\AppData\Local\Temp\ipykernel_3744\692399001.py:303: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  last_price = float(df_now.iloc[-1]["Close"])


[2025-09-21 09:08:22 UTC] Last close = 140.9000
 t+01: ret=+0.000% -> price ≈ 140.9000
 t+02: ret=+0.059% -> price ≈ 140.9828
 t+03: ret=+0.089% -> price ≈ 141.0257
 t+04: ret=+0.286% -> price ≈ 141.3035
 t+05: ret=+0.400% -> price ≈ 141.4629
 t+06: ret=-0.021% -> price ≈ 140.8711
 t+07: ret=+0.828% -> price ≈ 142.0670
 t+08: ret=+0.120% -> price ≈ 141.0691
 t+09: ret=+0.364% -> price ≈ 141.4135
 t+10: ret=+0.913% -> price ≈ 142.1858
 t+11: ret=+0.561% -> price ≈ 141.6909
 t+12: ret=+0.589% -> price ≈ 141.7304


Exception ignored from cffi callback <function buffer_callback at 0x0000019F9525E660>:
Traceback (most recent call last):
  File "C:\Users\chara\AppData\Local\Programs\Python\Python313\Lib\site-packages\curl_cffi\curl.py", line 100, in buffer_callback
    @ffi.def_extern()
KeyboardInterrupt: 

1 Failed download:
['ASHOKLEY.BO']: RequestException('Failed to perform, curl: (23) Failure writing output to destination, passed 13 returned 0. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Not enough recent data; sleeping...


In [4]:
#!/usr/bin/env python3
"""
predict_dwm.py (Daily, Weekly, Monthly)

- Uses daily bars ("1d") for long-term history.
- Creates features from a look-back window and targets for 1-day, 5-day (weekly), and 21-day (monthly) returns.
- Trains a MultiOutputRegressor to predict these three horizons simultaneously.
- Runs once to generate the latest prediction and then exits.
"""

import os
from datetime import datetime, timezone

import joblib
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score

# --------------- CONFIG ----------------
TICKER = "ASHOKLEY.BO"
INTERVAL = "1d"      # Changed to daily data
PERIOD = "10y"       # Changed to a longer period for more context
WINDOW = 60          # Look-back window in days
HORIZONS = [1, 5, 21] # Prediction horizons: 1-day, 5-day (1 week), 21-day (1 month)
PAST_RETURNS_LENGTH = 30 # Define a fixed length for past returns feature

# --- File Paths ---
MODEL_FILE = f"model_{TICKER.lower()}_dwm.joblib"
SCALER_FILE = f"scaler_{TICKER.lower()}_dwm.joblib"
LOG_FILE = "logs/predictions_dwm.csv"
os.makedirs("logs", exist_ok=True)
# ---------------------------------------

def fetch_history(ticker: str, interval: str, period: str) -> pd.DataFrame:
    """Fetches and cleans historical market data."""
    print(f"Fetching {period} of {interval} data for {ticker}...", flush=True)
    try:
        df = yf.download(
            tickers=ticker,
            interval=interval,
            period=period,
            progress=False,
            threads=False,
            auto_adjust=False
        )
        if df is None or df.empty:
            print("No data returned from yfinance.", flush=True)
            return pd.DataFrame()

        if not isinstance(df.index, pd.DatetimeIndex):
            df.index = pd.to_datetime(df.index)

        cols = ["Open", "High", "Low", "Close", "Volume"]
        df = df.loc[:, cols]
        for c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
        df.dropna(inplace=True)

        print(f"Successfully fetched {len(df)} rows of data.", flush=True)
        return df

    except Exception as e:
        print(f"An error occurred during data fetch: {e}", flush=True)
        return pd.DataFrame()

def build_features_and_targets(df: pd.DataFrame, window: int, horizons: list):
    print("Building features and targets...", flush=True)
    X_list, y_list = [], []

    for i in range(window, len(df) - max(horizons)):
        window_slice = df.iloc[i - window : i]
        close_prices = window_slice["Close"]
        volume = window_slice["Volume"]

        past_returns = close_prices.pct_change().dropna().values
        if past_returns.ndim > 1:
            past_returns = past_returns.flatten()

        mean_return = np.mean(past_returns)
        std_return = np.std(past_returns)
        mean_volume = np.mean(volume)

        current_date = df.index[i]
        day_of_week = current_date.dayofweek / 6.0
        month_of_year = current_date.month / 12.0

        padded_past_returns = np.zeros(PAST_RETURNS_LENGTH)
        actual_returns_length = min(len(past_returns), PAST_RETURNS_LENGTH)
        padded_past_returns[-actual_returns_length:] = past_returns[-actual_returns_length:]

        features = np.array([
            mean_return,
            std_return,
            mean_volume,
            day_of_week,
            month_of_year,
            *padded_past_returns
        ])

        current_price = df.iloc[i]["Close"]
        future_returns = []
        for h in horizons:
            if i + h >= len(df):
                future_returns.append(np.nan)
            else:
                future_price = df.iloc[i + h]["Close"]
                future_return = (future_price - current_price) / (current_price + 1e-9)
                future_returns.append(future_return)

        if not any(np.isnan(future_returns)):
             X_list.append(features)
             y_list.append(future_returns)

    X = np.array(X_list)
    y = np.array(y_list)
    y = y.reshape(y.shape[0], -1)
    print(f"Built X, y shapes: {X.shape}, {y.shape}", flush=True)
    return X, y

def train_and_save(X, y):
    print("Training new model...", flush=True)
    if X.shape[0] != y.shape[0]:
        raise RuntimeError(f"Shape mismatch: X={X.shape[0]}, y={y.shape[0]}")

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, shuffle=False)

    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_val_s = scaler.transform(X_val)

    base_estimator = GradientBoostingRegressor(n_estimators=150, learning_rate=0.05, max_depth=5, random_state=42)
    model = MultiOutputRegressor(base_estimator, n_jobs=-1)
    model.fit(X_train_s, y_train)

    y_pred = model.predict(X_val_s)
    for i, h in enumerate(HORIZONS):
        mae = mean_absolute_error(y_val[:, i], y_pred[:, i])
        r2 = r2_score(y_val[:, i], y_pred[:, i])
        print(f"  - Horizon {h}-day => Validation MAE: {mae:.6f} | R²: {r2:.4f}", flush=True)

    joblib.dump(model, MODEL_FILE)
    joblib.dump(scaler, SCALER_FILE)
    print(f"Saved model to {MODEL_FILE} and scaler to {SCALER_FILE}.", flush=True)
    return model, scaler

def load_model_and_scaler():
    if os.path.exists(MODEL_FILE) and os.path.exists(SCALER_FILE):
        print("Loading existing model and scaler from disk.", flush=True)
        model = joblib.load(MODEL_FILE)
        scaler = joblib.load(SCALER_FILE)
        return model, scaler
    return None, None

def make_prediction_vector(df_recent: pd.DataFrame, window: int):
    if len(df_recent) < window:
        print("Not enough recent data to create a feature vector.", flush=True)
        return None

    window_slice = df_recent.iloc[-window:]
    close_prices = window_slice["Close"]
    volume = window_slice["Volume"]

    past_returns = close_prices.pct_change().dropna().values
    if past_returns.ndim > 1:
        past_returns = past_returns.flatten()

    mean_return = np.mean(past_returns)
    std_return = np.std(past_returns)
    mean_volume = np.mean(volume)

    current_date = df_recent.index[-1]
    day_of_week = current_date.dayofweek / 6.0
    month_of_year = current_date.month / 12.0

    padded_past_returns = np.zeros(PAST_RETURNS_LENGTH)
    actual_returns_length = min(len(past_returns), PAST_RETURNS_LENGTH)
    padded_past_returns[-actual_returns_length:] = past_returns[-actual_returns_length:]

    features = np.array([
        mean_return,
        std_return,
        mean_volume,
        day_of_week,
        month_of_year,
        *padded_past_returns
    ]).reshape(1, -1)

    return features

def log_prediction(timestamp, ticker, pred_returns, last_price):
    row = {
        "timestamp": timestamp,
        "ticker": ticker,
        "last_price": float(last_price),
    }
    for i, h in enumerate(HORIZONS):
        ret = pred_returns[i]
        row[f"pred_ret_{h}d"] = float(ret)
        row[f"pred_price_{h}d"] = float(last_price * (1 + ret))

    df_log = pd.DataFrame([row])
    header = not os.path.exists(LOG_FILE)
    df_log.to_csv(LOG_FILE, mode="a", header=header, index=False)
    print(f"Prediction logged to {LOG_FILE}", flush=True)

def main():
    model, scaler = load_model_and_scaler()

    if model is None or scaler is None:
        df_hist = fetch_history(TICKER, INTERVAL, PERIOD)
        if df_hist.empty:
            print("Cannot train model without historical data. Exiting.", flush=True)
            return

        X, y = build_features_and_targets(df_hist, WINDOW, HORIZONS)
        if X.size == 0:
            print("Failed to build features. Exiting.", flush=True)
            return

        model, scaler = train_and_save(X, y)

    print("\nFetching latest data for prediction...", flush=True)
    df_recent = fetch_history(TICKER, INTERVAL, period=f"{WINDOW + max(HORIZONS) + 5}d")
    if df_recent.empty or len(df_recent) < WINDOW + max(HORIZONS):
        print("Not enough recent data to make a prediction. Exiting.", flush=True)
        return

    feature_vector = make_prediction_vector(df_recent, WINDOW)
    if feature_vector is None:
        print("Could not create feature vector. Exiting.", flush=True)
        return

    try:
        fv_scaled = scaler.transform(feature_vector)
        pred_returns = model.predict(fv_scaled)[0]
    except Exception as e:
        print(f"Error during prediction: {e}", flush=True)
        return

    last_close = df_recent.iloc[-1]["Close"]
    now_utc = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S %Z")

    print("\n" + "="*50, flush=True)
    print(f"PREDICTION for {TICKER} @ {now_utc}", flush=True)
    print(f"Based on last close price: {float(last_close):.2f}", flush=True)
    print("="*50, flush=True)

    names = ["Daily", "Weekly (5d)", "Monthly (21d)"]
    for i, h in enumerate(HORIZONS):
        ret = pred_returns[i]
        pred_price = float(last_close) * (1 + ret)
        print(f" {names[i]:<15} | Horizon: {h:2d} days | Return: {ret*100:+.2f}% | Predicted Price: {pred_price:.2f}", flush=True)

    log_prediction(now_utc, TICKER, pred_returns, float(last_close))

    print("\n--- Final Predictions ---", flush=True)
    print(f"Daily Predicted Close Price: {float(last_close) * (1 + pred_returns[0]):.2f}", flush=True)
    print(f"Weekly Predicted Close Price: {float(last_close) * (1 + pred_returns[1]):.2f}", flush=True)
    print(f"Monthly Predicted Close Price: {float(last_close) * (1 + pred_returns[2]):.2f}", flush=True)

if __name__ == "__main__":
    main()

Fetching 10y of 1d data for ASHOKLEY.BO...
Successfully fetched 2468 rows of data.
Building features and targets...
Built X, y shapes: (2387, 35), (2387, 3)
Training new model...
  - Horizon 1-day => Validation MAE: 0.015818 | R²: -0.0789
  - Horizon 5-day => Validation MAE: 0.039251 | R²: -0.3788
  - Horizon 21-day => Validation MAE: 0.115511 | R²: -2.1610
Saved model to model_ashokley.bo_dwm.joblib and scaler to scaler_ashokley.bo_dwm.joblib.

Fetching latest data for prediction...
Fetching 86d of 1d data for ASHOKLEY.BO...
Successfully fetched 85 rows of data.

PREDICTION for ASHOKLEY.BO @ 2025-09-21 09:35:27 UTC
Based on last close price: 140.90
 Daily           | Horizon:  1 days | Return: +1.16% | Predicted Price: 142.54
 Weekly (5d)     | Horizon:  5 days | Return: +3.88% | Predicted Price: 146.36
 Monthly (21d)   | Horizon: 21 days | Return: -2.39% | Predicted Price: 137.53
Prediction logged to logs/predictions_dwm.csv

--- Final Predictions ---
Daily Predicted Close Price: 142

C:\Users\chara\AppData\Local\Temp\ipykernel_18956\88424051.py:250: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  print(f"Based on last close price: {float(last_close):.2f}", flush=True)
C:\Users\chara\AppData\Local\Temp\ipykernel_18956\88424051.py:256: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  pred_price = float(last_close) * (1 + ret)
C:\Users\chara\AppData\Local\Temp\ipykernel_18956\88424051.py:259: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  log_prediction(now_utc, TICKER, pred_returns, float(last_close))
C:\Users\chara\AppData\Local\Temp\ipykernel_18956\88424051.py:262: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use 

In [5]:
#!/usr/bin/env python3
"""
predict_dwm.py (Daily, Weekly, Monthly)

- Uses daily bars ("1d") for long-term history.
- Creates features from a look-back window and targets for 1-day, 5-day (weekly), and 21-day (monthly) returns.
- Trains a MultiOutputRegressor to predict these three horizons simultaneously.
- Runs once to generate the latest prediction and then exits.
"""

import os
from datetime import datetime, timezone

import joblib
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score

# --------------- CONFIG ----------------
TICKER = "TATAMOTORS.BO"
INTERVAL = "1d"      # Changed to daily data
PERIOD = "10y"       # Changed to a longer period for more context
WINDOW = 60          # Look-back window in days
HORIZONS = [1, 5, 21] # Prediction horizons: 1-day, 5-day (1 week), 21-day (1 month)
PAST_RETURNS_LENGTH = 30 # Define a fixed length for past returns feature

# --- File Paths ---
MODEL_FILE = f"model_{TICKER.lower()}_dwm.joblib"
SCALER_FILE = f"scaler_{TICKER.lower()}_dwm.joblib"
LOG_FILE = "logs/predictions_dwm.csv"
os.makedirs("logs", exist_ok=True)
# ---------------------------------------

def fetch_history(ticker: str, interval: str, period: str) -> pd.DataFrame:
    """Fetches and cleans historical market data."""
    print(f"Fetching {period} of {interval} data for {ticker}...", flush=True)
    try:
        df = yf.download(
            tickers=ticker,
            interval=interval,
            period=period,
            progress=False,
            threads=False,
            auto_adjust=False
        )
        if df is None or df.empty:
            print("No data returned from yfinance.", flush=True)
            return pd.DataFrame()

        if not isinstance(df.index, pd.DatetimeIndex):
            df.index = pd.to_datetime(df.index)

        cols = ["Open", "High", "Low", "Close", "Volume"]
        df = df.loc[:, cols]
        for c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
        df.dropna(inplace=True)

        print(f"Successfully fetched {len(df)} rows of data.", flush=True)
        return df

    except Exception as e:
        print(f"An error occurred during data fetch: {e}", flush=True)
        return pd.DataFrame()

def build_features_and_targets(df: pd.DataFrame, window: int, horizons: list):
    print("Building features and targets...", flush=True)
    X_list, y_list = [], []

    for i in range(window, len(df) - max(horizons)):
        window_slice = df.iloc[i - window : i]
        close_prices = window_slice["Close"]
        volume = window_slice["Volume"]

        past_returns = close_prices.pct_change().dropna().values
        if past_returns.ndim > 1:
            past_returns = past_returns.flatten()

        mean_return = np.mean(past_returns)
        std_return = np.std(past_returns)
        mean_volume = np.mean(volume)

        current_date = df.index[i]
        day_of_week = current_date.dayofweek / 6.0
        month_of_year = current_date.month / 12.0

        padded_past_returns = np.zeros(PAST_RETURNS_LENGTH)
        actual_returns_length = min(len(past_returns), PAST_RETURNS_LENGTH)
        padded_past_returns[-actual_returns_length:] = past_returns[-actual_returns_length:]

        features = np.array([
            mean_return,
            std_return,
            mean_volume,
            day_of_week,
            month_of_year,
            *padded_past_returns
        ])

        current_price = df.iloc[i]["Close"]
        future_returns = []
        for h in horizons:
            if i + h >= len(df):
                future_returns.append(np.nan)
            else:
                future_price = df.iloc[i + h]["Close"]
                future_return = (future_price - current_price) / (current_price + 1e-9)
                future_returns.append(future_return)

        if not any(np.isnan(future_returns)):
             X_list.append(features)
             y_list.append(future_returns)

    X = np.array(X_list)
    y = np.array(y_list)
    y = y.reshape(y.shape[0], -1)
    print(f"Built X, y shapes: {X.shape}, {y.shape}", flush=True)
    return X, y

def train_and_save(X, y):
    print("Training new model...", flush=True)
    if X.shape[0] != y.shape[0]:
        raise RuntimeError(f"Shape mismatch: X={X.shape[0]}, y={y.shape[0]}")

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, shuffle=False)

    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_val_s = scaler.transform(X_val)

    base_estimator = GradientBoostingRegressor(n_estimators=150, learning_rate=0.05, max_depth=5, random_state=42)
    model = MultiOutputRegressor(base_estimator, n_jobs=-1)
    model.fit(X_train_s, y_train)

    y_pred = model.predict(X_val_s)
    for i, h in enumerate(HORIZONS):
        mae = mean_absolute_error(y_val[:, i], y_pred[:, i])
        r2 = r2_score(y_val[:, i], y_pred[:, i])
        print(f"  - Horizon {h}-day => Validation MAE: {mae:.6f} | R²: {r2:.4f}", flush=True)

    joblib.dump(model, MODEL_FILE)
    joblib.dump(scaler, SCALER_FILE)
    print(f"Saved model to {MODEL_FILE} and scaler to {SCALER_FILE}.", flush=True)
    return model, scaler

def load_model_and_scaler():
    if os.path.exists(MODEL_FILE) and os.path.exists(SCALER_FILE):
        print("Loading existing model and scaler from disk.", flush=True)
        model = joblib.load(MODEL_FILE)
        scaler = joblib.load(SCALER_FILE)
        return model, scaler
    return None, None

def make_prediction_vector(df_recent: pd.DataFrame, window: int):
    if len(df_recent) < window:
        print("Not enough recent data to create a feature vector.", flush=True)
        return None

    window_slice = df_recent.iloc[-window:]
    close_prices = window_slice["Close"]
    volume = window_slice["Volume"]

    past_returns = close_prices.pct_change().dropna().values
    if past_returns.ndim > 1:
        past_returns = past_returns.flatten()

    mean_return = np.mean(past_returns)
    std_return = np.std(past_returns)
    mean_volume = np.mean(volume)

    current_date = df_recent.index[-1]
    day_of_week = current_date.dayofweek / 6.0
    month_of_year = current_date.month / 12.0

    padded_past_returns = np.zeros(PAST_RETURNS_LENGTH)
    actual_returns_length = min(len(past_returns), PAST_RETURNS_LENGTH)
    padded_past_returns[-actual_returns_length:] = past_returns[-actual_returns_length:]

    features = np.array([
        mean_return,
        std_return,
        mean_volume,
        day_of_week,
        month_of_year,
        *padded_past_returns
    ]).reshape(1, -1)

    return features

def log_prediction(timestamp, ticker, pred_returns, last_price):
    row = {
        "timestamp": timestamp,
        "ticker": ticker,
        "last_price": float(last_price),
    }
    for i, h in enumerate(HORIZONS):
        ret = pred_returns[i]
        row[f"pred_ret_{h}d"] = float(ret)
        row[f"pred_price_{h}d"] = float(last_price * (1 + ret))

    df_log = pd.DataFrame([row])
    header = not os.path.exists(LOG_FILE)
    df_log.to_csv(LOG_FILE, mode="a", header=header, index=False)
    print(f"Prediction logged to {LOG_FILE}", flush=True)

def main():
    model, scaler = load_model_and_scaler()

    if model is None or scaler is None:
        df_hist = fetch_history(TICKER, INTERVAL, PERIOD)
        if df_hist.empty:
            print("Cannot train model without historical data. Exiting.", flush=True)
            return

        X, y = build_features_and_targets(df_hist, WINDOW, HORIZONS)
        if X.size == 0:
            print("Failed to build features. Exiting.", flush=True)
            return

        model, scaler = train_and_save(X, y)

    print("\nFetching latest data for prediction...", flush=True)
    df_recent = fetch_history(TICKER, INTERVAL, period=f"{WINDOW + max(HORIZONS) + 5}d")
    if df_recent.empty or len(df_recent) < WINDOW + max(HORIZONS):
        print("Not enough recent data to make a prediction. Exiting.", flush=True)
        return

    feature_vector = make_prediction_vector(df_recent, WINDOW)
    if feature_vector is None:
        print("Could not create feature vector. Exiting.", flush=True)
        return

    try:
        fv_scaled = scaler.transform(feature_vector)
        pred_returns = model.predict(fv_scaled)[0]
    except Exception as e:
        print(f"Error during prediction: {e}", flush=True)
        return

    last_close = df_recent.iloc[-1]["Close"]
    now_utc = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S %Z")

    print("\n" + "="*50, flush=True)
    print(f"PREDICTION for {TICKER} @ {now_utc}", flush=True)
    print(f"Based on last close price: {float(last_close):.2f}", flush=True)
    print("="*50, flush=True)

    names = ["Daily", "Weekly (5d)", "Monthly (21d)"]
    for i, h in enumerate(HORIZONS):
        ret = pred_returns[i]
        pred_price = float(last_close) * (1 + ret)
        print(f" {names[i]:<15} | Horizon: {h:2d} days | Return: {ret*100:+.2f}% | Predicted Price: {pred_price:.2f}", flush=True)

    log_prediction(now_utc, TICKER, pred_returns, float(last_close))

    print("\n--- Final Predictions ---", flush=True)
    print(f"Daily Predicted Close Price: {float(last_close) * (1 + pred_returns[0]):.2f}", flush=True)
    print(f"Weekly Predicted Close Price: {float(last_close) * (1 + pred_returns[1]):.2f}", flush=True)
    print(f"Monthly Predicted Close Price: {float(last_close) * (1 + pred_returns[2]):.2f}", flush=True)

if __name__ == "__main__":
    main()

Fetching 10y of 1d data for TATAMOTORS.BO...
Successfully fetched 2443 rows of data.
Building features and targets...
Built X, y shapes: (2362, 35), (2362, 3)
Training new model...
  - Horizon 1-day => Validation MAE: 0.014971 | R²: -0.0968
  - Horizon 5-day => Validation MAE: 0.035312 | R²: -0.0594
  - Horizon 21-day => Validation MAE: 0.063527 | R²: -0.0829
Saved model to model_tatamotors.bo_dwm.joblib and scaler to scaler_tatamotors.bo_dwm.joblib.

Fetching latest data for prediction...
Fetching 86d of 1d data for TATAMOTORS.BO...
Successfully fetched 85 rows of data.

PREDICTION for TATAMOTORS.BO @ 2025-09-21 09:37:38 UTC
Based on last close price: 708.05
 Daily           | Horizon:  1 days | Return: +0.14% | Predicted Price: 709.06
 Weekly (5d)     | Horizon:  5 days | Return: -1.21% | Predicted Price: 699.49
 Monthly (21d)   | Horizon: 21 days | Return: +0.52% | Predicted Price: 711.70
Prediction logged to logs/predictions_dwm.csv

--- Final Predictions ---
Daily Predicted Close 

C:\Users\chara\AppData\Local\Temp\ipykernel_18956\3508052989.py:250: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  print(f"Based on last close price: {float(last_close):.2f}", flush=True)
C:\Users\chara\AppData\Local\Temp\ipykernel_18956\3508052989.py:256: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  pred_price = float(last_close) * (1 + ret)
C:\Users\chara\AppData\Local\Temp\ipykernel_18956\3508052989.py:259: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  log_prediction(now_utc, TICKER, pred_returns, float(last_close))
C:\Users\chara\AppData\Local\Temp\ipykernel_18956\3508052989.py:262: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the futu

In [ ]:
# OUTPUT STORED IN EXCEL

In [7]:
pip install openpyxl


   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpy

In [1]:
#!/usr/bin/env python3
"""
predict_dwm.py (Daily, Weekly, Monthly)

- Uses daily bars ("1d") for long-term history.
- Creates features from a look-back window and targets for 1-day, 5-day (weekly), and 21-day (monthly) returns.
- Trains a MultiOutputRegressor to predict these three horizons simultaneously.
- Runs once to generate the latest prediction and then exits.
"""

import os
from datetime import datetime, timezone

import joblib
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
import openpyxl

# --------------- CONFIG ----------------
TICKER = "TATAMOTORS.BO"
INTERVAL = "1d"      # Daily data
PERIOD = "10y"       # Longer history
WINDOW = 60          # Look-back window in days
HORIZONS = [1, 5, 21] # Prediction horizons
PAST_RETURNS_LENGTH = 30

# --- File Paths ---
MODEL_FILE = f"model_{TICKER.lower()}_dwm.joblib"
SCALER_FILE = f"scaler_{TICKER.lower()}_dwm.joblib"
LOG_FILE = "logs/predictions_dwm.csv"
EXCEL_FILE = "logs/predictions_dwm.xlsx"
os.makedirs("logs", exist_ok=True)
# ---------------------------------------

def fetch_history(ticker: str, interval: str, period: str) -> pd.DataFrame:
    print(f"Fetching {period} of {interval} data for {ticker}...", flush=True)
    try:
        df = yf.download(
            tickers=ticker,
            interval=interval,
            period=period,
            progress=False,
            threads=False,
            auto_adjust=False
        )
        if df is None or df.empty:
            print("No data returned from yfinance.", flush=True)
            return pd.DataFrame()

        if not isinstance(df.index, pd.DatetimeIndex):
            df.index = pd.to_datetime(df.index)

        cols = ["Open", "High", "Low", "Close", "Volume"]
        df = df.loc[:, cols]
        for c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
        df.dropna(inplace=True)

        print(f"Successfully fetched {len(df)} rows of data.", flush=True)
        return df

    except Exception as e:
        print(f"An error occurred during data fetch: {e}", flush=True)
        return pd.DataFrame()

def build_features_and_targets(df: pd.DataFrame, window: int, horizons: list):
    print("Building features and targets...", flush=True)
    X_list, y_list = [], []

    for i in range(window, len(df) - max(horizons)):
        window_slice = df.iloc[i - window : i]
        close_prices = window_slice["Close"]
        volume = window_slice["Volume"]

        past_returns = close_prices.pct_change().dropna().values
        if past_returns.ndim > 1:
            past_returns = past_returns.flatten()

        mean_return = np.mean(past_returns)
        std_return = np.std(past_returns)
        mean_volume = np.mean(volume)

        current_date = df.index[i]
        day_of_week = current_date.dayofweek / 6.0
        month_of_year = current_date.month / 12.0

        padded_past_returns = np.zeros(PAST_RETURNS_LENGTH)
        actual_returns_length = min(len(past_returns), PAST_RETURNS_LENGTH)
        padded_past_returns[-actual_returns_length:] = past_returns[-actual_returns_length:]

        features = np.array([
            mean_return,
            std_return,
            mean_volume,
            day_of_week,
            month_of_year,
            *padded_past_returns
        ])

        current_price = df.iloc[i]["Close"]
        future_returns = []
        for h in horizons:
            if i + h >= len(df):
                future_returns.append(np.nan)
            else:
                future_price = df.iloc[i + h]["Close"]
                future_return = (future_price - current_price) / (current_price + 1e-9)
                future_returns.append(future_return)

        if not any(np.isnan(future_returns)):
            X_list.append(features)
            y_list.append(future_returns)

    X = np.array(X_list)
    y = np.array(y_list)
    y = y.reshape(y.shape[0], -1)
    print(f"Built X, y shapes: {X.shape}, {y.shape}", flush=True)
    return X, y

def train_and_save(X, y):
    print("Training new model...", flush=True)
    if X.shape[0] != y.shape[0]:
        raise RuntimeError(f"Shape mismatch: X={X.shape[0]}, y={y.shape[0]}")

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, shuffle=False)

    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_val_s = scaler.transform(X_val)

    base_estimator = GradientBoostingRegressor(n_estimators=150, learning_rate=0.05, max_depth=5, random_state=42)
    model = MultiOutputRegressor(base_estimator, n_jobs=-1)
    model.fit(X_train_s, y_train)

    y_pred = model.predict(X_val_s)
    for i, h in enumerate(HORIZONS):
        mae = mean_absolute_error(y_val[:, i], y_pred[:, i])
        r2 = r2_score(y_val[:, i], y_pred[:, i])
        print(f"  - Horizon {h}-day => Validation MAE: {mae:.6f} | R²: {r2:.4f}", flush=True)

    joblib.dump(model, MODEL_FILE)
    joblib.dump(scaler, SCALER_FILE)
    print(f"Saved model to {MODEL_FILE} and scaler to {SCALER_FILE}.", flush=True)
    return model, scaler

def load_model_and_scaler():
    if os.path.exists(MODEL_FILE) and os.path.exists(SCALER_FILE):
        print("Loading existing model and scaler from disk.", flush=True)
        model = joblib.load(MODEL_FILE)
        scaler = joblib.load(SCALER_FILE)
        return model, scaler
    return None, None

def make_prediction_vector(df_recent: pd.DataFrame, window: int):
    if len(df_recent) < window:
        print("Not enough recent data to create a feature vector.", flush=True)
        return None

    window_slice = df_recent.iloc[-window:]
    close_prices = window_slice["Close"]
    volume = window_slice["Volume"]

    past_returns = close_prices.pct_change().dropna().values
    if past_returns.ndim > 1:
        past_returns = past_returns.flatten()

    mean_return = np.mean(past_returns)
    std_return = np.std(past_returns)
    mean_volume = np.mean(volume)

    current_date = df_recent.index[-1]
    day_of_week = current_date.dayofweek / 6.0
    month_of_year = current_date.month / 12.0

    padded_past_returns = np.zeros(PAST_RETURNS_LENGTH)
    actual_returns_length = min(len(past_returns), PAST_RETURNS_LENGTH)
    padded_past_returns[-actual_returns_length:] = past_returns[-actual_returns_length:]

    features = np.array([
        mean_return,
        std_return,
        mean_volume,
        day_of_week,
        month_of_year,
        *padded_past_returns
    ]).reshape(1, -1)

    return features

def log_prediction(timestamp, ticker, pred_returns, last_price):
    row = {
        "timestamp": timestamp,
        "ticker": ticker,
        "last_price": float(last_price),
    }
    for i, h in enumerate(HORIZONS):
        ret = pred_returns[i]
        row[f"pred_ret_{h}d"] = float(ret)
        row[f"pred_price_{h}d"] = float(last_price * (1 + ret))

    df_log = pd.DataFrame([row])
    header = not os.path.exists(LOG_FILE)
    df_log.to_csv(LOG_FILE, mode="a", header=header, index=False)
    print(f"Prediction logged to {LOG_FILE}", flush=True)

def save_to_excel(ticker, actual_prices, predicted_prices):
    """Save predictions to Excel with company name, actuals, and predictions."""
    row = {
        "Company": ticker,
        "Actual_Daily": actual_prices[0],
        "Actual_Weekly": actual_prices[1],
        "Actual_Monthly": actual_prices[2],
        "Predicted_Daily": predicted_prices[0],
        "Predicted_Weekly": predicted_prices[1],
        "Predicted_Monthly": predicted_prices[2],
    }

    df_row = pd.DataFrame([row])

    if not os.path.exists(EXCEL_FILE):
        df_row.to_excel(EXCEL_FILE, index=False, engine="openpyxl")
    else:
        with pd.ExcelWriter(EXCEL_FILE, mode="a", engine="openpyxl", if_sheet_exists="overlay") as writer:
            startrow = writer.sheets["Sheet1"].max_row
            df_row.to_excel(writer, index=False, header=False, startrow=startrow)
    print(f"Prediction saved to {EXCEL_FILE}", flush=True)

def main():
    model, scaler = load_model_and_scaler()

    if model is None or scaler is None:
        df_hist = fetch_history(TICKER, INTERVAL, PERIOD)
        if df_hist.empty:
            print("Cannot train model without historical data. Exiting.", flush=True)
            return

        X, y = build_features_and_targets(df_hist, WINDOW, HORIZONS)
        if X.size == 0:
            print("Failed to build features. Exiting.", flush=True)
            return

        model, scaler = train_and_save(X, y)

    print("\nFetching latest data for prediction...", flush=True)
    df_recent = fetch_history(TICKER, INTERVAL, period=f"{WINDOW + max(HORIZONS) + 5}d")
    if df_recent.empty or len(df_recent) < WINDOW + max(HORIZONS):
        print("Not enough recent data to make a prediction. Exiting.", flush=True)
        return

    feature_vector = make_prediction_vector(df_recent, WINDOW)
    if feature_vector is None:
        print("Could not create feature vector. Exiting.", flush=True)
        return

    try:
        fv_scaled = scaler.transform(feature_vector)
        pred_returns = model.predict(fv_scaled)[0]
    except Exception as e:
        print(f"Error during prediction: {e}", flush=True)
        return

    last_close = df_recent.iloc[-1]["Close"]
    now_utc = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S %Z")

    print("\n" + "="*50, flush=True)
    print(f"PREDICTION for {TICKER} @ {now_utc}", flush=True)
    print(f"Based on last close price: {float(last_close):.2f}", flush=True)
    print("="*50, flush=True)

    names = ["Daily", "Weekly (5d)", "Monthly (21d)"]
    for i, h in enumerate(HORIZONS):
        ret = pred_returns[i]
        pred_price = float(last_close) * (1 + ret)
        print(f" {names[i]:<15} | Horizon: {h:2d} days | Return: {ret*100:+.2f}% | Predicted Price: {pred_price:.2f}", flush=True)

    log_prediction(now_utc, TICKER, pred_returns, float(last_close))

    # Collect actual prices for daily, weekly, monthly
    actual_prices = []
    for h in HORIZONS:
        if len(df_recent) >= h:
            actual_price = float(df_recent.iloc[-h]["Close"])
        else:
            actual_price = float(last_close)
        actual_prices.append(actual_price)

    predicted_prices = [
        float(last_close) * (1 + pred_returns[0]),
        float(last_close) * (1 + pred_returns[1]),
        float(last_close) * (1 + pred_returns[2]),
    ]

    save_to_excel(TICKER, actual_prices, predicted_prices)

    print("\n--- Final Predictions ---", flush=True)
    print(f"Daily Predicted Close Price: {predicted_prices[0]:.2f}", flush=True)
    print(f"Weekly Predicted Close Price: {predicted_prices[1]:.2f}", flush=True)
    print(f"Monthly Predicted Close Price: {predicted_prices[2]:.2f}", flush=True)

if __name__ == "__main__":
    main()


Loading existing model and scaler from disk.

Fetching latest data for prediction...
Fetching 86d of 1d data for TATAMOTORS.BO...
Successfully fetched 85 rows of data.

PREDICTION for TATAMOTORS.BO @ 2025-09-21 10:04:55 UTC
Based on last close price: 708.05
 Daily           | Horizon:  1 days | Return: +0.14% | Predicted Price: 709.06
 Weekly (5d)     | Horizon:  5 days | Return: -1.21% | Predicted Price: 699.49
 Monthly (21d)   | Horizon: 21 days | Return: +0.52% | Predicted Price: 711.70
Prediction logged to logs/predictions_dwm.csv
Prediction saved to logs/predictions_dwm.xlsx

--- Final Predictions ---
Daily Predicted Close Price: 709.06
Weekly Predicted Close Price: 699.49
Monthly Predicted Close Price: 711.70


C:\Users\chara\AppData\Local\Temp\ipykernel_4116\1876882424.py:273: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  print(f"Based on last close price: {float(last_close):.2f}", flush=True)
C:\Users\chara\AppData\Local\Temp\ipykernel_4116\1876882424.py:279: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  pred_price = float(last_close) * (1 + ret)
C:\Users\chara\AppData\Local\Temp\ipykernel_4116\1876882424.py:282: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  log_prediction(now_utc, TICKER, pred_returns, float(last_close))
C:\Users\chara\AppData\Local\Temp\ipykernel_4116\1876882424.py:288: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. 